In [45]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import os
import time
import numpy as np
site = "https://llvm.org/docs/LangRef.html"
driver = webdriver.Chrome()
driver.get(site)
initdir = os.getcwd()+'/'
libz_ = '../libs/'
el_synt = lambda synt: '\t\t'+synt.text.splitlines()[0]+'\n'+'\n'.join(list(map(lambda x : "\t\t\t"+x,synt.text.splitlines()[1:])))
splt_w_scope = r'\s\:*\=*\s*'
scope = r'\[[^\[\]]+\]'
from tqdm import tqdm

In [46]:
def constr_exp(obj,id1,id2):
    r_obj = []
    ind = 0 
    c_id1 = 0 
    c_id2 = 0
    c_el = ''
    while(ind!=len(obj)):
        if id1 in obj[ind]:
            c_id1+=re.findall(id1,obj[ind]).__len__()
            if id2 in obj[ind]:
                c_id2+=re.findall(id2,obj[ind]).__len__()
            c_el+=obj[ind]+' '
            if c_id1==c_id2:
                r_obj.append(c_el[:-1])
                c_id1 = 0
                c_id2 = 0
                c_el = ''
        elif id2 in obj[ind]:
            c_id2+=re.findall(id2,obj[ind]).__len__()
            c_el+=obj[ind]+' '
            if c_id1==c_id2:
                r_obj.append(c_el[:-1])
                c_id1 = 0
                c_id2 = 0
                c_el = ''
        elif c_id1!=c_id2:
            c_el+=obj[ind]+' '
        else:
            r_obj.append(obj[ind])
        ind+=1  
    return r_obj 

In [53]:
container = driver.find_element_by_id("instruction-reference")
with open(initdir+'pass.dmb','wt') as file:
    for section in tqdm.tqdm(container.find_elements_by_xpath(".//div[contains(@id,'instructions') or contains(@id,'operations')]")):
        file.write(section.get_attribute('id')+'\n')
        for instr in section.find_elements_by_xpath(".//div[contains(@id,'-instruction') or contains(@id,'-operation')]"):
            file.write('\t'+instr.get_attribute('id')+'\n')
            synt = instr.find_element_by_xpath(".//a[contains(text(),'Syntax:')]").find_element_by_xpath('../..')
            file.write(el_synt(synt)+'\n')
            try:
                args = instr.find_element_by_xpath(".//a[contains(text(),'Arguments:')]").find_element_by_xpath('../..')
                file.write(el_synt(args)+'\n')
            except Exception as e:
                file.write('\t\tArguments:\n')
                print('\t'+instr.get_attribute('id')+'\n')
            ovr = instr.find_element_by_xpath(".//a[contains(text(),'Overview:')]").find_element_by_xpath('../..')
            file.write(el_synt(ovr)+'\n')
            try:
                exp = instr.find_element_by_xpath(".//a[contains(text(),'Example:') or contains(text(),'Examples:')]").find_element_by_xpath('../..')
                file.write(el_synt(exp)+'\n')
            except Exception as e:
                file.write('\t\tExample:\n')
                print('\t'+instr.get_attribute('id')+'\n')
    

 11%|█         | 1/9 [00:04<00:32,  4.06s/it]

	unreachable-instruction

	unreachable-instruction



100%|██████████| 9/9 [00:24<00:00,  2.76s/it]


In [54]:
inst_gp = {}
curr_gp = ""
curr_inst = ""
ind = 0
with open(initdir+'pass.dmb','r') as file:
    obj = file.readlines()
    while(ind!=len(obj)):
        if re.match(r"[^\t]+",obj[ind]):
            inst_gp[obj[ind][:-1]] = {}
            curr_gp = obj[ind][:-1]
        elif re.match(r"\t\tSyntax\:",obj[ind]):
                inst_gp[curr_gp][curr_inst]['synt'] = []
                while(re.match('\t\t\t.*',obj[ind+1])):
                    ind+=1
                    inst_gp[curr_gp][curr_inst]['synt'].append(obj[ind][3:-1])
        elif re.match(r"\t\tArguments\:",obj[ind]):
                inst_gp[curr_gp][curr_inst]['args'] = []
                while(re.match('\t\t\t.*',obj[ind+1])):
                    ind+=1
                    inst_gp[curr_gp][curr_inst]['args'].append(obj[ind][3:-1])
        elif re.match(r"\t\tOverview\:",obj[ind]):
                inst_gp[curr_gp][curr_inst]['overview'] = []
                while(re.match('\t\t\t.*',obj[ind+1])):
                    ind+=1
                    inst_gp[curr_gp][curr_inst]['overview'].append(obj[ind][3:-1])
        elif re.match(r"\t\tExamples*\:",obj[ind]):
                inst_gp[curr_gp][curr_inst]['exm'] = []
                while((ind+1)<len(obj) and re.match('\t\t\t.*',obj[ind+1])):
                    ind+=1
                    inst_gp[curr_gp][curr_inst]['exm'].append(obj[ind][3:-1])
        elif re.match(r"\t.+",obj[ind]):
            inst_gp[curr_gp][obj[ind][1:-1]] = {}
            curr_inst = obj[ind][1:-1]
        ind+=1            

In [55]:
print(list(inst_gp.keys()).__len__())
for line in list(inst_gp.keys()):
    print(line)

9
terminator-instructions
unary-operations
binary-operations
bitwise-binary-operations
vector-operations
aggregate-operations
memory-access-and-addressing-operations
conversion-operations
other-operations


In [65]:
projections = {}
for prj in inst_gp:
    projections[prj] = {}
    for inst,det in inst_gp[prj].items():
            projections[prj][inst] = {'pttrn':[]}
            for sline in det['synt']:
                sline = re.sub(r'\s+',' ',sline)
                sline = re.split(r'\s*;\s*',sline )[0]
                projections[prj][inst]['opt'] = re.findall(r'\[.[^\[\]]*\]\**',sline)
                sline = re.sub(r'\[.[^\[\]]*\]\**','',sline)
                sline = re.sub(r'\s+',' ',sline)
                
                prj_ = np.array(re.split(splt_w_scope,sline))
                if '=' in sline:
                    np.put(prj_,[0,1],[prj_[1],prj_[0]])
                prj_ = prj_.tolist()    
                for t,r in [('\<','\>'),('\{','\}'),('\(','\)')]:
                    prj_ = constr_exp(prj_,t,r)
                projections[prj][inst]['pttrn']+=[prj_]
#                 print(sline)
#                 print(prj_)
                



## Instruction group relation

In [66]:
gp_projection = {}
for gp_ in projections:
    for pj_ in projections[gp_]:
        gp_projection[pj_] = {'group':gp_}
        if projections[gp_][pj_]['pttrn'].__len__()>1:
            print(pj_)
        gp_projection[pj_]['ptr'] = projections[gp_][pj_]['pttrn']
        gp_projection[pj_]['opt'] = projections[gp_][pj_]['opt']
# gp_projection

ret-instruction
br-instruction
invoke-instruction
callbr-instruction
catchswitch-instruction
cleanupret-instruction
add-instruction
sub-instruction
mul-instruction
udiv-instruction
sdiv-instruction
shl-instruction
lshr-instruction
ashr-instruction
extractelement-instruction
insertelement-instruction
shufflevector-instruction
load-instruction
store-instruction
getelementptr-instruction
select-instruction
call-instruction
landingpad-instruction


In [ ]:
def check_correctness(ptrn):
    to_check = [('>','<'),('(',')'),('{','}')]
    for each in ptrn:
        lem = np.array(list(each))
        sign,count = np.unique(lem,return_counts=True)
        ex = dict(zip(sign,count))
        for r_,l_ in to_check:
            if r_ in each and l_ in each:
                lem = np.array(list(each))
                sign,count = np.unique(lem,return_counts=True)
                ex = dict(zip(sign,count))
                if ex[r_]!=ex[l_]:
                    print(each)
            elif reduce(lambda x,y: x or y,list(map(lambda x:x in )),FAlse)


In [12]:
a = list('asdaa')
a = np.array(a)
lem,count = np.unique(a,return_counts=True)

{'a': 3, 'd': 1, 's': 1}

In [6]:
import numpy as np
a= np.array('a','s')
np.unique(a,return_counts=True)

(array(['asdasdasd'], dtype='<U9'), array([1]))

In [61]:
initdir+libz_+'pass_struct.dmp'

'/home/dope/Documents/thesis/thesis/DataScience-Scripts/../libs/pass_struct.dmp'

In [69]:
passes_info = {'pass-group':gp_projection,'group-pass':projections,'pass-info':inst_gp}
pickle.dump(passes_info,open(initdir+libz_+'pass_struct.dmp','wb'))